In [1]:
import warnings
warnings.filterwarnings('ignore')

In [1]:
import numpy as np
import tensorflow as tf
import itertools as it
import scipy.io.wavfile as wavfile

tf.enable_eager_execution()

#testData = tf.data.TFRecordDataset("data/nsynth-test.tfrecord")
res = tf.python_io.tf_record_iterator("data/nsynth-test.tfrecord")

i = 0
for e in res:
    print("\nRECORD ", i, "\n")
    e = tf.train.Example.FromString(e)
    d = dict(e.features.feature)
    keys = d.keys()
    for key in keys:
        if key == 'audio':
            print(key)
            print('omitted\n')
            #print(d.get(key))
            #wavfile.read(d.get(key))
        else:
            print(key)
            print (d.get(key))
    
    i += 1
    if i > 0:
        break

/home/ec2-user/anaconda3/envs/magenta-gpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ec2-user/anaconda3/envs/magenta-gpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ec2-user/anaconda3/envs/magenta-gpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ec2-user/anaconda3/


RECORD  0 

instrument_family
int64_list {
  value: 0
}

note_str
bytes_list {
  value: "bass_synthetic_033-100-100"
}

sample_rate
int64_list {
  value: 16000
}

instrument_source
int64_list {
  value: 2
}

qualities
int64_list {
  value: 0
  value: 0
  value: 0
  value: 0
  value: 0
  value: 0
  value: 0
  value: 0
  value: 0
  value: 0
}

instrument_source_str
bytes_list {
  value: "synthetic"
}

audio
omitted

instrument
int64_list {
  value: 417
}

qualities_str
bytes_list {
}

note
int64_list {
  value: 149013
}

instrument_str
bytes_list {
  value: "bass_synthetic_033"
}

instrument_family_str
bytes_list {
  value: "bass"
}

velocity
int64_list {
  value: 100
}

pitch
int64_list {
  value: 100
}



In [4]:
#     features = {
#         "note_str": tf.FixedLenFeature([], dtype=tf.string),
#         "pitch": tf.FixedLenFeature([1], dtype=tf.int64),
#         "velocity": tf.FixedLenFeature([1], dtype=tf.int64),
#         "audio": tf.FixedLenFeature([64000], dtype=tf.float32),
#         "qualities": tf.FixedLenFeature([10], dtype=tf.int64),
#         "instrument_source": tf.FixedLenFeature([1], dtype=tf.int64),
#         "instrument_family": tf.FixedLenFeature([1], dtype=tf.int64),
#     }

In [2]:
import warnings
import tensorflow as tf
import math
#from magenta.models.nsynth import utils
#from magenta.models.nsynth.wavenet import fastgen

from keras import layers
from keras import models
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam
import keras.backend as K
import librosa
import librosa.display
import pylab
import matplotlib.pyplot as plt
from utils import audio_utils, general_utils
from matplotlib import figure
import gc
from path import Path

import importlib

warnings.filterwarnings('ignore')


Using TensorFlow backend.


In [ ]:
importlib.reload(audio_utils)
importlib.reload(general_utils)

#audio_utils.write_spectograms('data/nsynth-test.tfrecord', 'data/nsynth-test-spectrograms')
audio_utils.write_spectograms_parallelized('data/nsynth-valid.tfrecord', 'data/nsynth-valid-spectrograms', 200)


processed 2450 files in 12 batches

In [13]:
import multiprocessing as mp
print("Number of processors: ", mp.cpu_count())

Number of processors:  4
